In [1]:
# stock_data.py

import alpaca
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.requests import StockLatestBarRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd

In [2]:
APCA_API_KEY_ID = 'PKD98H4EZF8YRDLUZ5I2'
APCA_API_SECRET_KEY = 'b5ZbNnx35m3uaB6vRnrU7TLQlgEfDw0bKf1Y8Zsm'
ticker = "SPY"
timeDiff = datetime.now() - relativedelta(hour=1)
tf = 5

In [3]:
# keys required for stock historical data client
client = StockHistoricalDataClient(APCA_API_KEY_ID, APCA_API_SECRET_KEY)

# multi symbol request - single symbol is similar
latest_bar_request_params = StockLatestBarRequest(
    symbol_or_symbols=[ticker],
    timeframe=TimeFrame(tf, alpaca.data.timeframe.TimeFrameUnit.Minute),)

latest_stock_bar = client.get_stock_latest_bar(latest_bar_request_params)
latest_stock_bar

{'SPY': {   'close': 438.43,
     'high': 438.47,
     'low': 438.43,
     'open': 438.47,
     'symbol': 'SPY',
     'timestamp': datetime.datetime(2023, 9, 20, 20, 3, tzinfo=datetime.timezone.utc),
     'trade_count': 3.0,
     'volume': 1200.0,
     'vwap': 438.449167}}

In [4]:
# keys required for stock historical data client
hClient = StockHistoricalDataClient(APCA_API_KEY_ID, APCA_API_SECRET_KEY)

# multi symbol request - single symbol is similar
historical_bar_request_params = StockBarsRequest(
    symbol_or_symbols=[ticker],
    timeframe=TimeFrame(tf, alpaca.data.timeframe.TimeFrameUnit.Minute),
    start=timeDiff)

historical_symbol_df = hClient.get_stock_bars(historical_bar_request_params).df
historical_symbol_df.reset_index(level='symbol', inplace=True)
historical_symbol_df.drop(columns=["symbol"], inplace=True)

historical_symbol_df

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2023-09-20 08:00:00+00:00,442.98,442.9800,442.72,442.72,2024.0,28.0,442.885479
2023-09-20 08:05:00+00:00,442.65,442.7800,442.65,442.78,1146.0,20.0,442.725855
2023-09-20 08:10:00+00:00,442.76,442.8800,442.71,442.71,1169.0,17.0,442.787690
2023-09-20 08:15:00+00:00,442.77,442.9900,442.76,442.99,6397.0,37.0,442.934617
2023-09-20 08:20:00+00:00,443.05,443.2500,443.05,443.25,6324.0,47.0,443.200768
...,...,...,...,...,...,...,...
2023-09-20 19:50:00+00:00,439.39,439.5700,439.13,439.18,2976028.0,17507.0,439.654348
2023-09-20 19:55:00+00:00,439.18,439.3600,438.43,438.58,6347410.0,34205.0,439.046426
2023-09-20 20:00:00+00:00,438.58,438.7600,438.22,438.37,1590268.0,3162.0,440.529418


In [5]:
def barToDF(bar, ticker, bigDF):
    df = pd.DataFrame(bar[ticker]).T
    df.columns = df.iloc[0]
    df = df.tail(-1)
    df.index = df["timestamp"]
    df = df.iloc[:, 2:]
    concatedDF = pd.concat([bigDF, df], axis=0)
    return concatedDF

In [6]:
resultingDF = barToDF(latest_stock_bar, "SPY", historical_symbol_df)
resultingDF

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2023-09-20 08:00:00+00:00,442.98,442.98,442.72,442.72,2024.0,28.0,442.885479
2023-09-20 08:05:00+00:00,442.65,442.78,442.65,442.78,1146.0,20.0,442.725855
2023-09-20 08:10:00+00:00,442.76,442.88,442.71,442.71,1169.0,17.0,442.78769
2023-09-20 08:15:00+00:00,442.77,442.99,442.76,442.99,6397.0,37.0,442.934617
2023-09-20 08:20:00+00:00,443.05,443.25,443.05,443.25,6324.0,47.0,443.200768
...,...,...,...,...,...,...,...
2023-09-20 19:55:00+00:00,439.18,439.36,438.43,438.58,6347410.0,34205.0,439.046426
2023-09-20 20:00:00+00:00,438.58,438.76,438.22,438.37,1590268.0,3162.0,440.529418
2023-09-20 20:05:00+00:00,438.37,438.5542,438.25,438.29,6192294.0,1906.0,438.702046


In [10]:
# imports
from datetime import datetime, timedelta

# Function to calculate the time until the next 5-minute interval
def calculate_time_until_next_interval():
    now = datetime.now()
    current_minute = now.minute
    seconds_until_next_interval = (5 - current_minute % 5) * 60 - now.second
    return timedelta(seconds=seconds_until_next_interval)

In [11]:
# Calculate the time until the next 5-minute interval
time_until_next_interval = calculate_time_until_next_interval()
time_until_next_interval

datetime.timedelta(seconds=118)

In [17]:
import asyncio

# await asyncio.sleep(time_until_next_interval.total_seconds())
await asyncio.sleep(10)

In [18]:
ls_bar = client.get_stock_latest_bar(latest_bar_request_params)
ls_bar

{'SPY': {   'close': 438.43,
     'high': 438.47,
     'low': 438.43,
     'open': 438.47,
     'symbol': 'SPY',
     'timestamp': datetime.datetime(2023, 9, 20, 20, 3, tzinfo=datetime.timezone.utc),
     'trade_count': 3.0,
     'volume': 1200.0,
     'vwap': 438.449167}}

In [22]:
resultingDF = barToDF(ls_bar, "SPY", resultingDF)
resultingDF

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2023-09-20 08:00:00+00:00,442.98,442.98,442.72,442.72,2024.0,28.0,442.885479
2023-09-20 08:05:00+00:00,442.65,442.78,442.65,442.78,1146.0,20.0,442.725855
2023-09-20 08:10:00+00:00,442.76,442.88,442.71,442.71,1169.0,17.0,442.78769
2023-09-20 08:15:00+00:00,442.77,442.99,442.76,442.99,6397.0,37.0,442.934617
2023-09-20 08:20:00+00:00,443.05,443.25,443.05,443.25,6324.0,47.0,443.200768
...,...,...,...,...,...,...,...
2023-09-20 20:05:00+00:00,438.37,438.5542,438.25,438.29,6192294.0,1906.0,438.702046
2023-09-20 20:10:00+00:00,438.64,438.64,438.19,438.26,1944214.0,1829.0,438.446815
2023-09-20 20:03:00+00:00,438.47,438.47,438.43,438.43,1200.0,3.0,438.449167


In [ ]:
l1 = loop.create_task(barToDF(latest_stock_bar, "SPY", historical_symbol_df))